In [8]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install selenium

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.7 MB 799.2 kB/s eta 0:00:12
   -- ------------------------------------- 0.5/9.7 MB 799.2 kB/s eta 0:00:12
   --- ------------------------------------ 0.8/9.7 MB 684.4 kB/s eta 0:00:14
   ---- ----------------------------------- 1.0/9.7 MB 671.0 kB/s eta 0:00:13
   ---- ----------------------------------- 1.0/9.7 MB 671.0 kB/s eta 0:00:13
   ---- ----------------------------------- 1.0/9.7 MB 671.0 kB/s eta 0:00:13
   ----- ---------------------------------- 1.3/9.7 MB 645.3 kB/s eta 0:00:13
   ----- ---------------------------------- 1.3/9.7 MB 645.3 kB/s eta 0:00:13
   ------ -----------------

In [5]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [6]:
# Path ke GeckoDriver, sesuaikan dengan path di komputermu
gecko_path = "D:/geckodriver.exe"  # Sesuaikan path ke GeckoDriver
service = Service(executable_path=gecko_path)

# Path ke Firefox executable
firefox_path = "C:/Program Files/Mozilla Firefox/firefox.exe"  # Lokasi Firefox.exe


In [7]:
# Konfigurasi Firefox
options = Options()
options.binary_location = firefox_path  # Menetapkan lokasi binary Firefox
options.headless = False  # Set ke True jika Anda tidak ingin membuka jendela browser

def create_driver():
    return webdriver.Firefox(service=service, options=options)


In [9]:

def get_tweet(element):
    try:
        user = element.find_element(By.XPATH, ".//*[contains(text(), '@')]").text
        text = element.find_element(By.XPATH, ".//div[@lang]").text
        date = element.find_element(By.XPATH, ".//time").get_attribute("datetime")

        try:
            reply_to_user = element.find_element(By.XPATH, ".//div[@dir='ltr']//a[contains(@href, '/')]").text
        except:
            reply_to_user = None

        try:
            tweet_link = element.find_element(By.XPATH, ".//a[@href and contains(@href, '/status/')]").get_attribute("href")
        except:
            tweet_link = None

        try:
            media_links = [media.get_attribute("src") for media in element.find_elements(By.XPATH, ".//img[contains(@src, 'media')]")]
        except:
            media_links = []

        tweet_data = [user, text, date, tweet_link, media_links, reply_to_user]
        return tweet_data
    except Exception as e:
        print(f"Error: {e}")
        return None

def click_retry_button(driver):
    try:
        retry_button = driver.find_element(By.XPATH, "//div[text()='Retry']/parent::div")
        retry_button.click()
        print("Clicked the retry button")
        WebDriverWait(driver, 15).until(EC.invisibility_of_element_located((By.XPATH, "//div[text()='Retry']")))
    except:
        pass

def scrape_tweets(driver):
    user_data = []
    text_data = []
    date_data = []
    tweet_link_data = []
    media_link_data = []
    reply_to_data = []
    tweet_ids = set()

    scrolling = True
    while scrolling and len(user_data) < 10000:
        try:
            print("Searching for tweets...")
            tweets = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.XPATH, "//article")))
            print(f"Number of tweets found: {len(tweets)}")
            for tweet in tweets:
                tweet_list = get_tweet(tweet)
                if tweet_list:
                    tweet_id = ''.join(tweet_list[:2])
                    if tweet_id not in tweet_ids:
                        tweet_ids.add(tweet_id)
                        user_data.append(tweet_list[0])
                        text_data.append(" ".join(tweet_list[1].split()))
                        date_data.append(tweet_list[2])
                        tweet_link_data.append(tweet_list[3])
                        media_link_data.append(tweet_list[4])
                        reply_to_data.append(tweet_list[5])

            last_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 30).until(lambda driver: driver.execute_script("return document.body.scrollHeight") > last_height)
            new_height = driver.execute_script("return document.body.scrollHeight")

            click_retry_button(driver)

            if new_height == last_height:
                scrolling = False
            else:
                last_height = new_height
                time.sleep(5)  # Tambahkan penundaan yang lebih lama

        except Exception as e:
            print(f"Error during scrolling or tweet retrieval: {e}")
            break

    return user_data, text_data, date_data, tweet_link_data, media_link_data, reply_to_data

driver = create_driver()
web = 'https://x.com/search?q=foodfess2%20tomoro%202024%20lang%3Aid%20until%3A2024-07-01%20since%3A2023-07-01&src=typed_query' # Masukan link yang akan di cari
driver.get(web)
time.sleep(15)
driver.save_screenshot('screenshot_before_search.png')

user_data, text_data, date_data, tweet_link_data, media_link_data, reply_to_data = scrape_tweets(driver)

driver.save_screenshot('screenshot_after_search.png')
driver.quit()

df = pd.DataFrame({
    'User': user_data,
    'Text': text_data,
    'Date': date_data,
    'Tweet Link': tweet_link_data,
    'Media Links': media_link_data,
    'Reply To': reply_to_data
})

print(df)

Searching for tweets...
Number of tweets found: 4
Searching for tweets...
Number of tweets found: 9
Searching for tweets...
Number of tweets found: 7
Searching for tweets...
Number of tweets found: 8
Searching for tweets...
Number of tweets found: 6
Searching for tweets...
Number of tweets found: 8
Searching for tweets...
Number of tweets found: 8
Searching for tweets...
Number of tweets found: 5
Error during scrolling or tweet retrieval: Message: 

                User                                               Text  \
0         @FOODFESS2  fess nyobain yg lagi hypeini matchanya tomoro ...   
1    @wwandaxvisionn  ODDIO “SARÀ PERCHÉ TI AMO” E “MAMMA MARIA” IL ...   
2         @FOODFESS2  tomoro nih kalo bentukannya orang udah gw nika...   
3   @EmiIiaSupremacy  BOTTOM TIER CHARACTER TOMOZAKI SEASON 2 AIRS T...   
4      @tomorrrro_56  #HADO ありがとうございました 僕とたろのコンビネーションはいかがだったでしょうか！笑 ...   
5   @EmiIiaSupremacy  HOLY SHIT, BOTTOM TIER CHARACTER TOMOZAKI SEAS...   
6       @TOMOO_staff 

In [10]:
import re

def extract_mentions(text):
    mentions = re.findall(r'@(\w+)', text)
    return mentions

df['Target'] = df['User']
df['Mentions'] = df['Text'].apply(extract_mentions)

df

,User,Text,Date,Tweet Link,Media Links,Reply To,Target,Mentions
0,@FOODFESS2,fess nyobain yg lagi hypeini matchanya tomoro ...,2024-02-18T01:01:52.000Z,https://x.com/FOODFESS2/status/175902037393135...,[https://pbs.twimg.com/media/GGlLtLlbUAA4IbH?f...,None,@FOODFESS2,[]
1,@wwandaxvisionn,ODDIO “SARÀ PERCHÉ TI AMO” E “MAMMA MARIA” IL ...,2024-02-09T21:37:49.000Z,https://x.com/wwandaxvisionn/status/1756069918...,[https://pbs.twimg.com/media/GF6_MlXXsAAPuh_?f...,¥le,@wwandaxvisionn,[]
2,@FOODFESS2,tomoro nih kalo bentukannya orang udah gw nika...,2024-06-13T07:45:12.000Z,https://x.com/FOODFESS2/status/180115886884596...,[https://pbs.twimg.com/media/GP8AcFgaIAA9gz5?f...,None,@FOODFESS2,[]
3,@EmiIiaSupremacy,BOTTOM TIER CHARACTER TOMOZAKI SEASON 2 AIRS T...,2024-02-28T15:09:38.000Z,https://x.com/EmiIiaSupremacy/status/176285759...,[https://pbs.twimg.com/media/GHbto87agAApBnj?f...,None,@EmiIiaSupremacy,[]
4,@tomorrrro_56,#HADO ありがとうございました 僕とたろのコンビネーションはいかがだったでしょうか！笑 ...,2024-02-03T13:26:39.000Z,https://x.com/tomorrrro_56/status/175377198740...,[https://pbs.twimg.com/media/GFamUqDagAAb4AY?f...,None,@tomorrrro_56,[]
5,@EmiIiaSupremacy,"HOLY SHIT, BOTTOM TIER CHARACTER TOMOZAKI SEAS...",2024-02-13T16:47:14.000Z,https://x.com/EmiIiaSupremacy/status/175744634...,[https://pbs.twimg.com/media/GGO0IWqbsAAmldf?f...,None,@EmiIiaSupremacy,[]
6,@TOMOO_staff,"TOMOO LIVE TOUR 2023-2024 ""TWO MOON"" FINAL公演無事...",2024-01-30T13:03:57.000Z,https://x.com/TOMOO_staff/status/1752316719761...,[https://pbs.twimg.com/media/GFF6R0zakAAcBV7?f...,None,@TOMOO_staff,[tarumikana]
7,@TOMOO_staff,象眠舎 Tour2024-UPCOMNIG- ありがとうございました 4月29日(月・祝) ...,2024-04-29T12:45:26.000Z,https://x.com/TOMOO_staff/status/1784926971174...,[https://pbs.twimg.com/media/GMVUznRaIAAtlMV?f...,None,@TOMOO_staff,[]
8,@EmiIiaSupremacy,"HOLY SHIT, BOTTOM TIER CHARACTER TOMOZAKI SEAS...",2024-01-30T16:55:13.000Z,https://x.com/EmiIiaSupremacy/status/175237492...,[https://pbs.twimg.com/media/GFGvs9AaEAAuMIl?f...,None,@EmiIiaSupremacy,[]
9,@EmiIiaSupremacy,"HOLY SHIT, BOTTOM TIER CHARACTER TOMOZAKI SEAS...",2024-01-23T17:44:24.000Z,https://x.com/EmiIiaSupremacy/status/174985058...,[https://pbs.twimg.com/media/GEi31K-bkAAx1TT?f...,None,@EmiIiaSupremacy,[]


In [11]:
# Simpan kedalam csv
df.to_csv('hasil-crawling.csv', index=False, sep=";")